In [28]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import cv2

from matplotlib import pyplot as plt
import tensorflow as tf
from tqdm import tqdm
from PIL import Image
from tensorflow import keras
from keras import Input
from keras.layers import Dense, Reshape, LeakyReLU, Conv2D, Conv2DTranspose, Flatten, Dropout
from keras.models import Model
from keras.optimizers import RMSprop

In [29]:
path = 'D:/Medical_Imaging/Medical_Imaging_Zusatz/BreaKHis_v1/Dataset_40X/benign/adenosis/'

WIDTH = 128
HEIGHT = 128

images = []
read = lambda imname: np.asarray(Image.open(imname).convert('L')) #'LA', 'L'

for image in os.listdir(path):

    img = read(path + "/" + image)
    # img = cv2.imread(path + folder + "/" + image)
    # img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img = cv2.resize(img, (HEIGHT, WIDTH))

    # täusche RGB Image an damit das fürs ResNet als input verwendet werden kann
    img = np.repeat(img[..., np.newaxis], 3, -1)

    images.append(np.array(img))

In [30]:
#Image shape
images = np.array(images) / 255
print(images.shape)

(114, 128, 128, 3)


In [31]:
LATENT_DIM = 32
CHANNELS = 3

def create_generator():
    
    gen_input = Input(shape=(LATENT_DIM, ))

    x = Dense(128 * 16 * 16)(gen_input)
    x = LeakyReLU()(x)
    x = Reshape((16, 16, 128))(x)

    x = Conv2D(256, 5, padding='same')(x)
    x = LeakyReLU()(x)

    x = Conv2DTranspose(256, 4, strides=2, padding='same')(x)
    x = LeakyReLU()(x)

    x = Conv2DTranspose(256, 4, strides=2, padding='same')(x)
    x = LeakyReLU()(x)

    x = Conv2DTranspose(256, 4, strides=2, padding='same')(x)
    x = LeakyReLU()(x)

    x = Conv2D(512, 5, padding='same')(x)
    x = LeakyReLU()(x)
    x = Conv2D(512, 5, padding='same')(x)
    x = LeakyReLU()(x)
    x = Conv2D(CHANNELS, 7, activation='tanh', padding='same')(x)

    generator = Model(gen_input, x)
    return generator

In [32]:
def create_discriminator():
    disc_input = Input(shape=(HEIGHT, WIDTH, CHANNELS))

    x = Conv2D(256, 3)(disc_input)
    x = LeakyReLU()(x)

    x = Conv2D(256, 4, strides=2)(x)
    x = LeakyReLU()(x)

    x = Conv2D(256, 4, strides=2)(x)
    x = LeakyReLU()(x)

    x = Conv2D(256, 4, strides=2)(x)
    x = LeakyReLU()(x)

    x = Conv2D(256, 4, strides=2)(x)
    x = LeakyReLU()(x)

    x = Flatten()(x)
    x = Dropout(0.4)(x)

    x = Dense(1, activation='sigmoid')(x)
    discriminator = Model(disc_input, x)

    optimizer = RMSprop(
        lr=.0001,
        clipvalue=1.0,
        decay=1e-8
    )

    discriminator.compile(
        optimizer=optimizer,
        loss='binary_crossentropy'
    )

    return discriminator

In [33]:
generator = create_generator()

In [38]:
generator.load_weights('D:/Medical_Imaging/Medical_Imaging_Zusatz/Saved_Checkpoints/gan2_40x_49999.h5')

ValueError: Layer count mismatch when loading weights from file. Model expected 8 layers, found 2 saved layers.

In [34]:
discriminator = create_discriminator()
discriminator.trainable = False

c:\Users\elias\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\optimizers\optimizer_v2\rmsprop.py:143: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [35]:
gan_input = Input(shape=(LATENT_DIM, ))
gan_output = discriminator(generator(gan_input))
gan = Model(gan_input, gan_output)

In [36]:
optimizer = RMSprop(lr = .0001, clipvalue = 1.0, decay = 1e-8)
gan.compile(optimizer=optimizer, loss='binary_crossentropy')

# gan.build(input_shape=(LATENT_DIM, ))
# 
model = gan.load_weights('D:/Medical_Imaging/Medical_Imaging_Zusatz/Saved_Checkpoints/gan2_40x_49999.h5')

In [24]:
iters = 15
batch_size = 16

In [37]:
model.summary()

AttributeError: 'NoneType' object has no attribute 'summary'

In [25]:
latent_vectors = np.random.normal(size = (batch_size, LATENT_DIM))
model.predict(latent_vectors)

AttributeError: 'NoneType' object has no attribute 'predict'

# Backlog

In [2]:
# Global Variables
NOISE_DIM = 100
NUM_EXAMPLES_TO_GENERATE = 500

In [3]:
# create generator class
class Generator(tf.keras.Model):

    def __init__(self):
        super(Generator, self).__init__()
        self.fc1 = tf.keras.layers.Dense(8 * 8 * 256, use_bias=False)
        self.batchnorm1 = tf.keras.layers.BatchNormalization()

        self.conv1 = tf.keras.layers.Conv2DTranspose(256 * 8, (4, 4), strides=(1, 1), padding='same', use_bias=False)
        self.batchnorm2 = tf.keras.layers.BatchNormalization()

        self.conv2 = tf.keras.layers.Conv2DTranspose(256 * 4, (4, 4), strides=(2, 2), padding='same', use_bias=False)
        self.batchnorm3 = tf.keras.layers.BatchNormalization()

        self.conv3 = tf.keras.layers.Conv2DTranspose(256 * 2, (4, 4), strides=(2, 2), padding='same', use_bias=False)
        self.batchnorm4 = tf.keras.layers.BatchNormalization()

        self.conv4 = tf.keras.layers.Conv2DTranspose(256 * 1, (4, 4), strides=(2, 2), padding='same', use_bias=False)
        self.batchnorm5 = tf.keras.layers.BatchNormalization()

        self.conv5 = tf.keras.layers.Conv2DTranspose(128, (4, 4), strides=(2, 2), padding='same', use_bias=False)
        self.batchnorm6 = tf.keras.layers.BatchNormalization()

        self.conv6 = tf.keras.layers.Conv2DTranspose(1, (4, 4), strides=(2, 2), padding='same', use_bias=False)

    def call(self, x, training=True):
        x = self.fc1(x)
        x = self.batchnorm1(x, training=training)
        x = tf.nn.relu(x)

        x = tf.reshape(x, shape=(-1, 8, 8, 256))

        x = self.conv1(x)
        x = self.batchnorm2(x, training=training)
        x = tf.nn.relu(x)

        x = self.conv2(x)
        x = self.batchnorm3(x, training=training)
        x = tf.nn.relu(x)

        x = self.conv3(x)
        x = self.batchnorm4(x, training=training)
        x = tf.nn.relu(x)

        x = self.conv4(x)
        x = self.batchnorm5(x, training=training)
        x = tf.nn.relu(x)

        x = self.conv5(x)
        x = self.batchnorm6(x, training=training)
        x = tf.nn.relu(x)

        x = tf.nn.tanh(self.conv6(x))

        return x


## Benign Types

### Adenosis

In [4]:
generator_adenosis = Generator()
generator_optimizer_adenosis = tf.keras.optimizers.legacy.Adam(1e-4)

checkpoint_path_adenosis = 'D:/Medical_Imaging_Zusatz/Saved_Checkpoints/benign/adenosis/'
save_dir_adenosis = "D:/Medical_Imaging_Zusatz/Gan_Images/benign/adenosis/"

In [5]:
checkpoint_adenosis = tf.train.Checkpoint(generator_optimizer = generator_optimizer_adenosis, generator = generator_adenosis)
checkpoint_adenosis.restore(tf.train.latest_checkpoint(checkpoint_path_adenosis))

# random vector for image generation 
random_vector_for_generation = tf.random.normal([NUM_EXAMPLES_TO_GENERATE, NOISE_DIM])

predictions = generator_adenosis(random_vector_for_generation, training = False)

for number, prediction in enumerate(predictions):

    fig = plt.figure(figsize = (8, 8))
    plt.imshow(prediction[ :, :, 0] * 127.5 + 127.5, cmap = 'gray')
    plt.axis('off')
    plt.savefig(os.path.join(save_dir_adenosis, 'gan_image_{}_{:04d}.png'.format('adenosis', number)))
    plt.close()

### Fibroadenoma

In [6]:
generator_fibroadenoma = Generator()
generator_optimizer_fibroadenoma = tf.keras.optimizers.legacy.Adam(1e-4)

checkpoint_path_fibroadenoma = 'D:/Medical_Imaging_Zusatz/Saved_Checkpoints/benign/fibroadenoma/'
save_dir_fibroadenoma = "D:/Medical_Imaging_Zusatz/Gan_Images/benign/fibroadenoma/"

In [8]:
checkpoint_fibroadenoma = tf.train.Checkpoint(generator_optimizer = generator_optimizer_fibroadenoma, generator = generator_fibroadenoma)
checkpoint_fibroadenoma.restore(tf.train.latest_checkpoint(checkpoint_path_fibroadenoma))

# random vector for image generation 
random_vector_for_generation = tf.random.normal([NUM_EXAMPLES_TO_GENERATE, NOISE_DIM])

predictions = generator_fibroadenoma(random_vector_for_generation, training = False)

for number, prediction in enumerate(predictions):

    fig = plt.figure(figsize = (8, 8))
    plt.imshow(prediction[ :, :, 0] * 127.5 + 127.5, cmap = 'gray')
    plt.axis('off')
    plt.savefig(os.path.join(save_dir_fibroadenoma, 'gan_image_{}_{:04d}.png'.format('fibroadenoma', number)))
    plt.close()

### Phyllodes_tumor

In [9]:
generator_phyllodes_tumor = Generator()
generator_optimizer_phyllodes_tumor = tf.keras.optimizers.legacy.Adam(1e-4)

checkpoint_path_phyllodes_tumor = 'D:/Medical_Imaging_Zusatz/Saved_Checkpoints/benign/phyllodes_tumor/'
save_dir_phyllodes_tumor = "D:/Medical_Imaging_Zusatz/Gan_Images/benign/phyllodes_tumor/"

In [10]:
checkpoint_phyllodes_tumor = tf.train.Checkpoint(generator_optimizer = generator_optimizer_phyllodes_tumor, generator = generator_phyllodes_tumor)
checkpoint_phyllodes_tumor.restore(tf.train.latest_checkpoint(checkpoint_path_phyllodes_tumor))

# random vector for image generation 
random_vector_for_generation = tf.random.normal([NUM_EXAMPLES_TO_GENERATE, NOISE_DIM])

predictions = generator_phyllodes_tumor(random_vector_for_generation, training = False)

for number, prediction in enumerate(predictions):

    fig = plt.figure(figsize = (8, 8))
    plt.imshow(prediction[ :, :, 0] * 127.5 + 127.5, cmap = 'gray')
    plt.axis('off')
    plt.savefig(os.path.join(save_dir_phyllodes_tumor, 'gan_image_{}_{:04d}.png'.format('phyllodes_tumor', number)))
    plt.close()

### Tubular_adenoma

In [11]:
generator_tubular_adenoma = Generator()
generator_optimizer_tubular_adenoma = tf.keras.optimizers.legacy.Adam(1e-4)

checkpoint_path_tubular_adenoma = 'D:/Medical_Imaging_Zusatz/Saved_Checkpoints/benign/tubular_adenoma/'
save_dir_tubular_adenoma = "D:/Medical_Imaging_Zusatz/Gan_Images/benign/tubular_adenoma/"

In [12]:
checkpoint_tubular_adenoma = tf.train.Checkpoint(generator_optimizer = generator_optimizer_tubular_adenoma, generator = generator_tubular_adenoma)
checkpoint_tubular_adenoma.restore(tf.train.latest_checkpoint(checkpoint_path_tubular_adenoma))

# random vector for image generation 
random_vector_for_generation = tf.random.normal([NUM_EXAMPLES_TO_GENERATE, NOISE_DIM])

predictions = generator_tubular_adenoma(random_vector_for_generation, training = False)

for number, prediction in enumerate(predictions):

    fig = plt.figure(figsize = (8, 8))
    plt.imshow(prediction[ :, :, 0] * 127.5 + 127.5, cmap = 'gray')
    plt.axis('off')
    plt.savefig(os.path.join(save_dir_tubular_adenoma, 'gan_image_{}_{:04d}.png'.format('tubular_adenoma', number)))
    plt.close()

## Malignant Images

### Ductal_carcinoma

In [16]:
generator_ductal_carcinoma = Generator()
generator_optimizer_ductal_carcinoma = tf.keras.optimizers.legacy.Adam(1e-4)

checkpoint_path_ductal_carcinoma = 'D:/Medical_Imaging_Zusatz/Saved_Checkpoints/malignant/ductal_carcinoma/'
save_dir_ductal_carcinoma = "D:/Medical_Imaging_Zusatz/Gan_Images/malignant/ductal_carcinoma/"

In [17]:
checkpoint_ductal_carcinoma = tf.train.Checkpoint(generator_optimizer = generator_optimizer_ductal_carcinoma, generator = generator_ductal_carcinoma)
checkpoint_ductal_carcinoma.restore(tf.train.latest_checkpoint(checkpoint_path_ductal_carcinoma))

# random vector for image generation 
random_vector_for_generation = tf.random.normal([NUM_EXAMPLES_TO_GENERATE, NOISE_DIM])

predictions = generator_ductal_carcinoma(random_vector_for_generation, training = False)

for number, prediction in enumerate(predictions):

    fig = plt.figure(figsize = (8, 8))
    plt.imshow(prediction[ :, :, 0] * 127.5 + 127.5, cmap = 'gray')
    plt.axis('off')
    plt.savefig(os.path.join(save_dir_ductal_carcinoma, 'gan_image_{}_{:04d}.png'.format('ductal_carcinoma', number)))
    plt.close()

### Lobular_carcinoma

In [18]:
generator_lobular_carcinoma = Generator()
generator_optimizer_lobular_carcinoma = tf.keras.optimizers.legacy.Adam(1e-4)

checkpoint_path_lobular_carcinoma = 'D:/Medical_Imaging_Zusatz/Saved_Checkpoints/malignant/lobular_carcinoma/'
save_dir_lobular_carcinoma = "D:/Medical_Imaging_Zusatz/Gan_Images/malignant/lobular_carcinoma/"

In [19]:
checkpoint_lobular_carcinoma = tf.train.Checkpoint(generator_optimizer = generator_optimizer_lobular_carcinoma, generator = generator_lobular_carcinoma)
checkpoint_lobular_carcinoma.restore(tf.train.latest_checkpoint(checkpoint_path_lobular_carcinoma))

# random vector for image generation 
random_vector_for_generation = tf.random.normal([NUM_EXAMPLES_TO_GENERATE, NOISE_DIM])

predictions = generator_lobular_carcinoma(random_vector_for_generation, training = False)

for number, prediction in enumerate(predictions):

    fig = plt.figure(figsize = (8, 8))
    plt.imshow(prediction[ :, :, 0] * 127.5 + 127.5, cmap = 'gray')
    plt.axis('off')
    plt.savefig(os.path.join(save_dir_lobular_carcinoma, 'gan_image_{}_{:04d}.png'.format('lobular_carcinoma', number)))
    plt.close()

### Mucinous_carcinoma

In [20]:
generator_mucinous_carcinoma = Generator()
generator_optimizer_mucinous_carcinoma = tf.keras.optimizers.legacy.Adam(1e-4)

checkpoint_path_mucinous_carcinoma = 'D:/Medical_Imaging_Zusatz/Saved_Checkpoints/malignant/mucinous_carcinoma/'
save_dir_mucinous_carcinoma = "D:/Medical_Imaging_Zusatz/Gan_Images/malignant/mucinous_carcinoma/"

In [21]:
checkpoint_mucinous_carcinoma = tf.train.Checkpoint(generator_optimizer = generator_optimizer_mucinous_carcinoma, generator = generator_mucinous_carcinoma)
checkpoint_mucinous_carcinoma.restore(tf.train.latest_checkpoint(checkpoint_path_mucinous_carcinoma))

# random vector for image generation 
random_vector_for_generation = tf.random.normal([NUM_EXAMPLES_TO_GENERATE, NOISE_DIM])

predictions = generator_mucinous_carcinoma(random_vector_for_generation, training = False)

for number, prediction in enumerate(predictions):

    fig = plt.figure(figsize = (8, 8))
    plt.imshow(prediction[ :, :, 0] * 127.5 + 127.5, cmap = 'gray')
    plt.axis('off')
    plt.savefig(os.path.join(save_dir_mucinous_carcinoma, 'gan_image_{}_{:04d}.png'.format('mucinous_carcinoma', number)))
    plt.close()

### Papillary_carcinoma

In [22]:
generator_papillary_carcinoma = Generator()
generator_optimizer_papillary_carcinoma = tf.keras.optimizers.legacy.Adam(1e-4)

checkpoint_path_papillary_carcinoma = 'D:/Medical_Imaging_Zusatz/Saved_Checkpoints/malignant/papillary_carcinoma/'
save_dir_papillary_carcinoma = "D:/Medical_Imaging_Zusatz/Gan_Images/malignant/papillary_carcinoma/"

In [23]:
checkpoint_papillary_carcinoma = tf.train.Checkpoint(generator_optimizer = generator_optimizer_papillary_carcinoma, generator = generator_papillary_carcinoma)
checkpoint_papillary_carcinoma.restore(tf.train.latest_checkpoint(checkpoint_path_papillary_carcinoma))

# random vector for image generation 
random_vector_for_generation = tf.random.normal([NUM_EXAMPLES_TO_GENERATE, NOISE_DIM])

predictions = generator_papillary_carcinoma(random_vector_for_generation, training = False)

for number, prediction in enumerate(predictions):

    fig = plt.figure(figsize = (8, 8))
    plt.imshow(prediction[ :, :, 0] * 127.5 + 127.5, cmap = 'gray')
    plt.axis('off')
    plt.savefig(os.path.join(save_dir_papillary_carcinoma, 'gan_image_{}_{:04d}.png'.format('papillary_carcinoma', number)))
    plt.close()

## Evaluate generated Images

## Backup

In [ ]:
# for index, checkpoint_path in enumerate(checkpoint_path_benign_array):

#     # print(checkpoint_path)

#     checkpoint = tf.train.Checkpoint(generator_optimizer = generator_optimizer, generator = generator)

#     checkpoint.restore(tf.train.latest_checkpoint(checkpoint_path))

#     # print(checkpoint)

#     # random vector for image generation 
#     random_vector_for_generation = tf.random.normal([NUM_EXAMPLES_TO_GENERATE, NOISE_DIM])

#     # print(random_vector_for_generation)

#     predictions = generator(random_vector_for_generation, training = False)

#     for number, prediction in enumerate(predictions):

#         fig = plt.figure(figsize=(8, 8))

#         #    for i in range(predictions.shape[0]):
#         #       plt.subplot(4, 4, i + 1)
#         plt.imshow(prediction[ :, :, 0] * 127.5 + 127.5, cmap = 'gray')
#         # plt.title(f'Generated image at epoch: {Temp}')
#         plt.axis('off')
#         # plt.close()
#         plt.savefig(os.path.join(save_dir_benign_array[index], 'GAN_Image_{}_{:04d}.png'.format(benign_types[index], number)))

## Temp Checkpoint until each cancer type gets checkpoint

In [ ]:
# temp_checkpoint_path = "D:/Medical_Imaging_Zusatz/Saved_Checkpoints/checkpoints/"

# temp_checkpoint_array = [temp_checkpoint_path, temp_checkpoint_path, temp_checkpoint_path, temp_checkpoint_path]
# temp_checkpoint_array

In [ ]:
# types_benign = utils.get_types_array("benign")

# print(types_benign)

In [ ]:
# checkpoint_path_benign = 'D:/Saved_Checkpoints/benign/'
# checkpoint_path_benign_array = utils.get_paths(checkpoint_path_benign, types_benign)

# print(checkpoint_path_benign_array)

In [ ]:
# #benign folders
# save_dir_benign = "D:/Medical_Imaging_Zusatz/Gan_Images/benign/"
# save_dir_benign_array = utils.get_paths(save_dir_benign, types_benign)

# print(save_dir_benign_array)

In [ ]:
# generator = utils.Generator()
# generator_optimizer = tf.keras.optimizers.legacy.Adam(1e-4)

In [ ]:
# utils.generate_sample_images(generator, generator_optimizer, checkpoint_path_benign_array[1], save_dir_benign_array[1], types_benign[1])